In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

Les packages ont été installés avec succès.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
]  # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",  # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Les modèles ont été importés et configurés avec succès.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Connexion à Google Drive réussie.

In [4]:
from datasets import load_dataset
# Load your preprocessed data
train_data_en = load_dataset("json", data_files="drive/train_data_3_en.json")
train_data_fr = load_dataset("json", data_files="drive/train_data_3_fr.json")
test_data_fr = load_dataset("json", data_files="drive/test_data_3_fr.json")
test_data_en = load_dataset("json", data_files="drive/test_data_3_en.json")

Les jeux de données ont été chargés avec succès.

In [5]:
def generate_data(train_data, lang):
    process_datas = []
    for section in train_data["train"]:
        if lang == "fr":
            questions = section["questions"]
            reponses = section["reponses"]
            textes = section["textes"]
            for j in range(len(questions)):
                combined_text = ""
                true_case_texte = textes[j][0]
                false_case_texte = textes[j][1]
                true_reponse = reponses[j][0]
                false_reponse = reponses[j][1]
                question = questions[j]
                process_datas.append({"instruction": f"{question}", "input":f"{true_reponse}", "output":f"{true_case_texte}"})
                process_datas.append({"instruction": f"{question}", "input":f"{false_reponse}", "output":f"{false_case_texte}"})
        else:
            questions = section["questions"]
            reponses = section["answers"]
            textes = section["texts"]
            for j in range(len(questions)):
                combined_text = ""
                true_case_texte = textes[j][0]
                false_case_texte = textes[j][1]
                true_reponse = reponses[j][0]
                false_reponse = reponses[j][1]
                question = questions[j]
                process_datas.append({"instruction": f"{question}", "input":f"{true_reponse}", "output":f"{true_case_texte}"})
                process_datas.append({"instruction": f"{question}", "input":f"{false_reponse}", "output":f"{false_case_texte}"})
    return process_datas

La fonction de génération de données a été définie avec succès.

In [6]:
datas = []
datas = datas + generate_data(train_data_fr, "fr")
datas = datas + generate_data(train_data_en, "en")

Les données ont été générées avec succès.

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,  # Increase to 2 for larger GPUs like A100/V100/16+GB
        gradient_accumulation_steps = 16,  # Increase to 32 for 16GB+ GPUs.
        max_steps = 100,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.03,
        fp16 = False,
        bf16 = True,
        report_to = "none",
        output_dir = "drive/sft-results",
    )
)

L'entraîneur a été configuré avec succès.

In [8]:
trainer.train()

L'entraînement a commencé avec succès.

In [9]:
from transformers import pipeline
finetuned_model = FastLanguageModel.from_pretrained("drive/sft-results")
qa_pipeline = pipeline("text-generation", model=finetuned_model, tokenizer=tokenizer)

# Testing the model on an example prompt
example_prompt = "Le locataire peut-il constaté l'état du logement ?"
response = qa_pipeline(example_prompt)
print(response)

Le modèle a été affiné et testé avec succès.